In [11]:
import pandas as pd
import torch
from torch import nn, optim, tensor
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader 
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
from pathlib import Path

In [13]:
dftraining = pd.read_csv(Path('./data/spaceship-titanic/train.csv'))
dftest = pd.read_csv(Path('./data/spaceship-titanic/test.csv'))

In [4]:
dftraining

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
dftraining.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [18]:
dftraining.fillna(dftraining.mode()).isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [19]:
cats = ['HomePlanet', 'Cabin', 'Destination']
conts = ['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [20]:
dftraining['Cabin'].str[0].unique()

array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object)

In [21]:
dftraining['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [22]:
def clean_df(df, fillmode='median'):
    df = df.drop(['PassengerId', 'Name'], axis = 1)
    
    df['Cabin'] = df['Cabin'].str[0]

    df = pd.get_dummies(df, columns=cats)
    
    fill_method = getattr(df, fillmode)
    df = df.fillna(fill_method())
    
    for cont in conts:
        df[cont] = np.log1p(df[cont].astype(float))
    
    
    return df

In [23]:
clean_df(dftraining).astype(float).head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,...,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0.0,3.688879,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,3.218876,0.000000,4.700480,2.302585,3.258097,6.309918,3.806662,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,0.0,4.077537,0.693147,3.784190,8.182280,0.000000,8.812248,3.912023,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,3.526361,0.000000,0.000000,7.157735,5.918894,8.110728,5.267858,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,2.833213,0.000000,5.717028,4.262680,5.023881,6.338594,1.098612,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [24]:
def split_df(df, split=0.8):
    train = df.sample(frac=split)
    valid = df.drop(train.index)
    return train, valid

In [25]:
dftrain, dfvalid = split_df(dftraining)

In [ ]:
class SpaceTitanic(Dataset):
    def __init__(self, df, fillmode='median', train=True, bs=32):
        self.train = train
        self.clean_ds = clean_df(df, fillmode)
        
        # For the testing dataset
        self.idx = 0
        
    def __getitem__(self, i):
        row = self.clean_ds.iloc[i].astype(np.float32)
        x = tensor(row.drop(labels=['Transported']))
        y = tensor(row.loc(axis=0)['Transported'])
        return x, y
        
    def __len__(self):
        return len(self.clean_ds)
    
    def __iter__(self):
        if not self.train:
            self.idx = 0
        return self
    
    def __next__(self):
        return 

In [27]:
class Block(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(n_in),
            nn.Linear(n_in, n_out),
            nn.ReLU(),
        )
        
    def forward(self, x):
        return self.net(x)

class SpaceTitanicModel(nn.Module):
    def __init__(self, n_in, n_out, n_h, n_hidden_layers=5):
        super().__init__()
        self.model = nn.Sequential(
            Block(n_in, n_h),
            *[Block(n_h, n_h) for i in range(n_hidden_layers)],
            Block(n_h, n_out)
        )
        
    def forward(self, x):
        return self.model(x)

In [28]:
dstrain, dsvalid = SpaceTitanic(dftrain), SpaceTitanic(dfvalid)

In [29]:
def normalize(tns): return (tns - tns.mean()) / tns.std()

In [30]:
class Learner():
    def __init__(self, model, dls, opt_fn=optim.SGD, lr = 1e-3, loss_fn=F.mse_loss):
        self.model = model
        self.dls = dls
        self.opt = opt_fn(model.parameters(), lr=lr)
        self.loss_fn = loss_fn
        
    def fit(self, epochs=1):
        for epoch in range(epochs):
            self._fit(True)
            torch.no_grad()(self._fit(False))                

                
    def _fit(self, train=True):
        self.model.train() if train else self.model.eval()
        dl = self.dls[0] if train else self.dls[1]
        
        for x, y in dl:
            x = normalize(x)
            pred = self.model(x)
            pred = pred.view(-1)
            loss = self.loss_fn(pred, y)
            loss.backward()
            self.opt.step()
            self.opt.zero_grad()
            text = 'Train: ' if train else 'Valid: '
            print(text + str(loss.item()))
        

In [49]:
class Metrics:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.vals, self.ns = [], []
        
    def add(self, inp, targets=None, n=1):
        self.last = self.calc(inp, targets)
        self.vals.append(self.last)
        self.ns.append(n)
    
    @property
    def value(self):
        ns = tensor(self.ns)
        return (tensor(self.vals) * ns).sum() / np.sum(self.ns)
    
    def calc(self, inp, targets):
        return inp

In [162]:
class Accuracy(Metrics):
    def __init__(self):
        super().__init__()
    
    def calc(self, inp, targets):
        return (inp == targets).float().mean()

In [115]:
bs = 32
dls = DataLoader(dstrain, batch_size=bs), DataLoader(dsvalid, batch_size=bs)
n_features = dstrain.clean_ds.shape[1] - 1
model = SpaceTitanicModel(n_features, 1, 50, 4)

In [116]:
learn = Learner(model, dls, loss_fn=F.binary_cross_entropy_with_logits, lr=0.005, opt_fn=optim.AdamW)

In [117]:
learn.fit()

Train: 0.7150667905807495
Train: 0.6879686713218689
Train: 0.6393744349479675
Train: 0.6268638372421265
Train: 0.7320060729980469
Train: 0.6348843574523926
Train: 0.6451891660690308
Train: 0.6652562022209167
Train: 0.5678857564926147
Train: 0.6060229539871216
Train: 0.6548645496368408
Train: 0.6692057847976685
Train: 0.5942496061325073
Train: 0.5373249053955078
Train: 0.6182397603988647
Train: 0.5703446269035339
Train: 0.5312290191650391
Train: 0.531898558139801
Train: 0.6232460737228394
Train: 0.5718412399291992
Train: 0.6408202052116394
Train: 0.5773100852966309
Train: 0.5241373181343079
Train: 0.5481441617012024
Train: 0.6069740056991577
Train: 0.6598124504089355
Train: 0.5784619450569153
Train: 0.5345971584320068
Train: 0.511218249797821
Train: 0.6628280878067017
Train: 0.6763116121292114
Train: 0.4807722866535187
Train: 0.5525473356246948
Train: 0.5197776556015015
Train: 0.4792449176311493
Train: 0.44751113653182983
Train: 0.4801584780216217
Train: 0.5614829659461975
Train: 0.4163

In [ ]:
class test_dl(dftest):

In [ ]:
tensor(clean_df(dftest).astype(np.float32).to_numpy()).shape

torch.Size([4277, 22])